# Introduction

API doc:https://binance-docs.github.io/apidocs/spot/en/

The Following API can return all historical trading related data of tokens activated in Binance enchanges. 

In [18]:
# Packages
import time
import json

import requests
import pandas as pd
import numpy as np
import datetime
from urllib.parse import urljoin, urlencode
from dateutil import parser


# Obtian the exchange info

In [20]:
# Obtain the exchange information firstly
def exchangeInfo():
    BASE_URL = 'https://api.binance.com'
    PATH = '/api/v3/exchangeInfo'
    
    url = urljoin(BASE_URL, PATH)
    
    response = requests.get(url)

    data = response.json()
    data = pd.DataFrame(data['symbols'])
    return data

In [21]:
tokenInfo = exchangeInfo()

tokenInfo.head()

,symbol,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,icebergAllowed,ocoAllowed,quoteOrderQtyMarketAllowed,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions
0,ETHBTC,TRADING,ETH,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
1,LTCBTC,TRADING,LTC,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
2,BNBBTC,TRADING,BNB,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
3,NEOBTC,TRADING,NEO,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
4,QTUMETH,TRADING,QTUM,8,ETH,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]


In [22]:
tokenInfo.to_csv("Exchange_info.csv")

# example
tokenInfo['symbol'][0]

'ETHBTC'

# obtain the historicl data of tokens

Binance exchange provides from monthly frequency data to minute frequency data.

In [23]:
# timedelta_fre = ['W','D','hours','m','s','ms']
itv = ["1m","3m","5m","15m","30m","1h","2h","4h","6h","8h","12h","1d","3d","1w","1M"]

# historical of token price should be indexed by its symbol.
smb = tokenInfo['symbol']

In [24]:
def start_end_time(startTime,interval,length = 1000):
#     Due to the limit of the api, we need a function to help cut a long time into small period.
#     The function input the start time and interval and return the start time and endtime. 
#     
#     read the time
    startTime = datetime.datetime.strptime(startTime, "%Y-%m-%d %H:%M:%S")
    
#     unit 
    unit = interval
#     transfer the interval frequence
    if unit in ["1m","3m","5m","15m","30m"]:
        unit = 'm'
    elif unit in ["1h","2h","4h","6h","8h","12h"]:
        unit = 'hours'
    elif unit in ["1d","3d"]:
        unit = 'D'
    else:
        unit = 'W'
        
    endTime = pd.to_datetime(startTime)+pd.to_timedelta(length, unit = unit)

    startTime = datetime.datetime.timestamp(startTime)*1000
    endTime = datetime.datetime.timestamp(endTime)*1000
    
    return str(int(startTime)),str(int(endTime))

In [25]:
def klines(symbol,startTime,endTime,interval):
#    This function is aimed to gather the historical price data.    
    BASE_URL = 'https://api.binance.com'
    PATH = '/api/v3/klines'
    
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime':startTime,
        'endTime':endTime,
        'limit':1000 # 500/1000 is the limit for each request
    }

    url = urljoin(BASE_URL, PATH)

    response = requests.get(url,params=params)

    data = response.json()
    data = pd.DataFrame(data)
#     change the cols
    col = ['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades',
       'Taker buy base asset volume','Taker buy quote asset volume','Ignore']
    
    if not data.empty:
        data.columns = col
#       transfer the timestamp into time
        data['Open time'] = pd.to_datetime(data['Open time'],unit='ms',utc=True)
        data['Close time'] = pd.to_datetime(data['Close time'],unit='ms',utc=True)
        return data
    else:
        print('No data')

# Sample of data

After build the function of API and exchange information, we can now obtain every token with every period data.

For example, we here use famous 1m BTC as our data sample.

In [26]:
smb[11]

'BTCUSDT'

In [28]:
itv[0]

'1m'

In [30]:
symbol = smb[11]
interval = itv[0]

startTime = '2021-09-01 00:00:00'
[startTime,endTime] = start_end_time(startTime=startTime,interval=interval)

data = klines(symbol=symbol,startTime=startTime,endTime=endTime,interval=interval)

In [31]:
data.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2021-09-01 04:00:00+00:00,47043.65000000,47054.78000000,46991.96000000,47009.38000000,16.04384000,2021-09-01 04:00:59.999000+00:00,754341.09833020,667,8.45966000,397743.66640310,0
1,2021-09-01 04:01:00+00:00,47009.38000000,47025.24000000,46980.02000000,46995.37000000,22.86129000,2021-09-01 04:01:59.999000+00:00,1074492.76913610,533,12.05659000,566711.30732960,0
2,2021-09-01 04:02:00+00:00,46995.37000000,47020.26000000,46969.56000000,46995.11000000,19.67009000,2021-09-01 04:02:59.999000+00:00,924338.18065390,456,9.16799000,430803.69030850,0
3,2021-09-01 04:03:00+00:00,46995.11000000,47020.00000000,46984.68000000,47010.03000000,12.12577000,2021-09-01 04:03:59.999000+00:00,569978.26374720,371,6.85744000,322347.16252190,0
4,2021-09-01 04:04:00+00:00,47010.02000000,47017.25000000,46985.33000000,46985.35000000,10.84569000,2021-09-01 04:04:59.999000+00:00,509797.02832680,426,6.61054000,310725.14683290,0


In [32]:
data.to_csv('BTC_1min.csv')